To replicate this notebook, you can find data here: http://ourairports.com/data/.
### References:  
HANA window functions: https://help.sap.com/viewer/4fe29514fd584807ac9f2a04f6754767/2.0.00/en-US/20a353327519101495dfd0a87060a0d3.html  
SQL vs pandas: http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html  
SQL to Pandas: https://medium.com/jbennetcodes/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e

In [4]:
import pandas as pd

airports = pd.read_csv('data/airports.csv',keep_default_na=False, na_values = ['']) # NA as north American
airport_freq = pd.read_csv('data/airport-frequencies.csv')

In [5]:
airports.head()

,id,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,home_link
0,6523,00A,heliport,Total Rf Heliport,11.0,NA,US,US-PA,Bensalem,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,3435.0,NA,US,US-KS,Leoti,NaN
2,6524,00AK,small_airport,Lowell Field,450.0,NA,US,US-AK,Anchor Point,NaN
3,6525,00AL,small_airport,Epps Airpark,820.0,NA,US,US-AL,Harvest,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NA,US,US-AR,Newport,NaN


In [68]:
airport_freq.head()

,id,airport_ref,airport_ident,type,description,frequency_mhz
0,70518,6528,00CA,CTAF,CTAF,122.9
1,307581,6589,01FL,ARCAL,NaN,122.9
2,75239,6589,01FL,CTAF,CEDAR KNOLL TRAFFIC,122.8
3,60191,6756,04CA,CTAF,CTAF,122.9
4,59287,6779,04MS,UNIC,UNICOM,122.8


# SQL Anatomy

`SELECT (DISTINCT) COLUMN_NAME 
        AGG() OVER (PARTITION BY .. ORDER BY ROW N and CURRENT ROW)
    FROM ...
    WHERE ...
    GROUPBY ...
    HAVING ...
`    

    
## SELECT, WHERE, DISTINCT, LIMIT 

|                      SQL                     |                 Pandas                |
|:--------------------------------------------:|:-------------------------------------:|
| select * from airports                       | airports                              |
| select * from airports limit 3               | airports.head(3)                      |
| select id from airports where ident = 'KLAX' | airports[airports.ident == 'KLAX'].id |
| select distinct type from airport            | airports.type.unique()                |

## SELECT with multiple conditions

`where A = 'x' and B >123 and ....`

|         SQL                       |                              Pandas                              |
|:---------------------------------:|:----------------------------------------------------------------:|
| select * from airports where iso_region = 'US-CA' and type = 'seaplane_base'                         | airports[(airports.iso_region == 'US-CA') & (airports.type == 'seaplane_base')]                                    |
| select ident, name, municipality from airports where iso_region = 'US-CA' and type = 'large_airport' | airports[(airports.iso_region == 'US-CA') & (airports.type == 'large_airport')][['ident', 'name', 'municipality']] |

In [69]:
airports[(airports.iso_region == 'US-CA') & 
         (airports.type == 'large_airport')][['ident', 'name', 'municipality']].head(3)

,ident,name,municipality
26072,KBAB,Beale Air Force Base,Marysville
26911,KEDW,Edwards Air Force Base,Edwards
27701,KLAX,Los Angeles International Airport,Los Angeles


## ORDER BY
By default, Pandas will sort things in ascending order. To reverse that, provide ascending=False.

|                  SQL               |                      Pandas                  |
|:----------------------------------:|:--------------------------------------------:|
|select * from airport_freq where airport_ident = 'KLAX' order by type| airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values('type')|
|select * from airport_freq where airport_ident = 'KLAX' order by airport_ref acse,type desc   | airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values(['airport_ref','type'], ascending=[True,False])|

In [70]:
airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values(['airport_ref','type'], ascending=[True,False]).head()

,id,airport_ref,airport_ident,type,description,frequency_mhz
11904,60776,3632,KLAX,UNIC,UNICOM,122.95
11903,60775,3632,KLAX,TWR,TWR,119.80
11902,60774,3632,KLAX,OPS,AF,37.22
11900,60772,3632,KLAX,MISC,CG,34.50
11901,60773,3632,KLAX,MISC,CG,898.40


## IN… NOT IN
We know how to filter on a value, but what about a list of values — IN condition? In pandas, .isin() operator works the same way. To negate any condition, use ~.


|                                  SQL                                 |                           Pandas                           |
|:--------------------------------------------------------------------:|:----------------------------------------------------------:|
| select * from airports where type in ('heliport', 'balloonport')     | airports[airports.type.isin(['heliport', 'balloonport'])]  |
| select * from airports where type not in ('heliport', 'balloonport') | airports[~airports.type.isin(['heliport', 'balloonport'])] |

In [72]:
airports[~airports.type.isin(['heliport', 'balloonport'])].head()

,id,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,home_link
1,323361,00AA,small_airport,Aero B Ranch Airport,3435.0,NA,US,US-KS,Leoti,NaN
2,6524,00AK,small_airport,Lowell Field,450.0,NA,US,US-AK,Anchor Point,NaN
3,6525,00AL,small_airport,Epps Airpark,820.0,NA,US,US-AL,Harvest,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NA,US,US-AR,Newport,NaN
5,322127,00AS,small_airport,Fulton Airport,1100.0,NA,US,US-OK,Alex,NaN


## GROUP BY, COUNT, ORDER BY
Grouping is straightforward: use the .groupby() operator. There’s a subtle difference between semantics of a COUNT in SQL and Pandas.  

**.count()** will return the number of non-null/NaN values.  
**.size()** will return the count of all values 

**.to_frame(col)** convert Series to DataFrame

|                                                           SQL                                                           |                                                                     Pandas                                                                    |
|:-----------------------------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------:|
| select iso_country, type, count(&ast;) from airports group by iso_country, type order by iso_country, type              | airports.groupby(['iso_country', 'type']).size()                                                                                              |
| select iso_country, type, count(&ast;) from airports group by iso_country, type order by iso_country, count(&ast;) desc | airports.groupby(['iso_country', 'type']).size().to_frame('size').reset_index().sort_values(['iso_country', 'size'], ascending=[True, False]) |

In [73]:
airports.groupby(['iso_country', 'type']).size().head(5)#.to_frame('size')

iso_country  type          
AD           heliport           2
AE           closed             3
             heliport          21
             large_airport      4
             medium_airport     7
dtype: int64

In [74]:
airports.groupby(['iso_country', 'type']).size().head(5).to_frame('size')

size
iso_country type                
AD          heliport           2
AE          closed             3
            heliport          21
            large_airport      4
            medium_airport     7

## HAVING
In SQL, you can additionally filter grouped data using a HAVING condition. In Pandas, you can use **.filter()** and provide a Python function (or a lambda) that will return True if the group should be included into the result.


|                                                                     SQL                                                                     |                                                                   Pandas                                                                   |
|:-------------------------------------------------------------------------------------------------------------------------------------------:|:------------------------------------------------------------------------------------------------------------------------------------------:|
| select type, count(&ast;) from airports where iso_country = 'US' group by type having count(&ast;) > 1000 order by count(&ast;) desc | airports[airports.iso_country == 'US'].groupby('type').filter(lambda g: len(g) > 1000).groupby('type').size().sort_values(ascending=False) |

In [75]:
airports[airports.iso_country == 'US'].groupby('type').size()

type
balloonport          18
closed             1149
heliport           6250
large_airport       170
medium_airport      692
seaplane_base       564
small_airport     13778
dtype: int64

In [76]:
# fliter the groups with more than 1000 rows, and group by type, count how many rows
airports[airports.iso_country == 'US'].groupby('type').filter(lambda rows : len(rows) > 1000).groupby('type').size()

type
closed            1149
heliport          6250
small_airport    13778
dtype: int64

In [92]:
airports[airports.iso_country == 'US'].groupby('type').size().to_frame('size').query('size > 1000')

,size
type,
closed,1149
heliport,6250
small_airport,13778


## Top N records

Let’s say we did some preliminary querying, and now have a dataframe called by_country, that contains the number of airports per country:

|                                    SQL                                   |                           Pandas                          |
|:------------------------------------------------------------------------:|:---------------------------------------------------------:|
| select iso_country from by_country order by size desc limit 10           | by_country.nlargest(10, columns='airport_count')          |
| select iso_country from by_country order by size desc limit 10 offset 10 | by_country.nlargest(20, columns='airport_count').tail(10) |


In [93]:
airports.groupby('iso_country').size().to_frame('counts').reset_index().nlargest(3, columns = 'counts')

,iso_country,counts
225,US,22621
29,BR,4320
35,CA,2782


## Missing values

|                                    SQL                                   |                           Pandas                          |
|:------------------------------------------------------------------------:|:---------------------------------------------------------:|
| select * from airports where home_link is not null           | airports[airports.home_link.notnull()]          |
| select * from airports where home_link is null           | airports[airports.home_link.isnull()]          |




In [10]:
airports[airports.home_link.notnull()].head(3)

,id,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,home_link
55,6570,00W,small_airport,Lower Granite State Airport,719.0,NA,US,US-WA,Colfax,http://www.wsdot.wa.gov/aviation/AllStateAirpo...
437,6924,07FA,small_airport,Ocean Reef Club Airport,8.0,NA,US,US-FL,Key Largo,https://www.oceanreef.com/community/private-ai...
690,315634,0D9,small_airport,Air Park North,1170.0,NA,US,US-MI,Alba,http://airparknorth.net/?page_id=6


## DISTINCT

Want to remove duplicate for results

|                                    SQL                                   |                           Pandas                          |
|:------------------------------------------------------------------------:|:---------------------------------------------------------:|
| select distinct continent, iso_country from airports            | airports[['continent','iso_country']].drop_duplicates()      |


In [23]:
airports[['continent','iso_country']].drop_duplicates().head()

,continent,iso_country
45425,SA,PE
44565,SA,BO
44173,SA,CO
40656,SA,AR
41165,SA,CL


In [22]:
airports[['continent','iso_country']].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

## Aggregate functions (MIN, MAX, MEAN)
Calculate min, max, mean, and median length of a runway:


|                                           SQL                                          |                            Pandas                            |
|:--------------------------------------------------------------------------------------:|:------------------------------------------------------------:|
| select max(elevation_ft), min(elevation_ft), mean(elevation_ft), median(elevation_ft) from airports | airports.agg({'elevation_ft': ['min', 'max', 'mean', 'median']}) |

In [78]:
airports.agg({'elevation_ft': ['min', 'max', 'mean', 'median']})

,elevation_ft
min,-1266.000000
max,22000.000000
mean,1236.467134
median,717.000000


In [79]:
airports.agg({'elevation_ft': ['min', 'max', 'mean', 'median']}).T

,min,max,mean,median
elevation_ft,-1266.0,22000.0,1236.467134,717.0


## INSERT, UPDATE

|                         SQL                        |                                    Pandas                                   |
|:--------------------------------------------------:|:---------------------------------------------------------------------------:|
| create table heroes (id integer, name text);       | df1 = pd.DataFrame({'id': [1, 2], 'name': ['Harry Potter', 'Ron Weasley']}) |
| insert into heroes values (1, 'Harry Potter');     | df2 = pd.DataFrame({'id': [3], 'name': ['Hermione Granger']})               |
| insert into heroes values (2, 'Ron Weasley');      |                                                                             |
| insert into heroes values (3, 'Hermione Granger'); | pd.concat([df1, df2]).reset_index(drop=True)                                |
| update airports set home_link = 'http://www.lawa.org/welcomelax.aspx' where ident == 'KLAX' | airports.loc[airports['ident'] == 'KLAX', 'home_link'] = 'http://www.lawa.org/welcomelax.aspx' |

## ADD, DELETE by Columns and Rows

|                    SQL                   |                         Pandas                         |
|:----------------------------------------:|:------------------------------------------------------:|
| alter airports add (abbrevation text)    | airports['abbrevation'] = 'None'                       |
|      -                                   | airports['abbrevation'] = airports['municipality'].str[:3]  |
| alter airports drop (abbrevation)        | airports.drop(['abbrevation'], axis=1)                 |
| delete from lax_freq where type = 'MISC' | lax_freq = lax_freq[lax_freq.type != 'MISC']           |
|                                    -     | lax_freq.drop(lax_freq[lax_freq.type == 'MISC'].index) |





In [80]:
airports['abbrevation'] = airports['municipality'].str[:3]
airports.head()

,id,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,home_link,abbrevation
0,6523,00A,heliport,Total Rf Heliport,11.0,NA,US,US-PA,Bensalem,NaN,Ben
1,323361,00AA,small_airport,Aero B Ranch Airport,3435.0,NA,US,US-KS,Leoti,NaN,Leo
2,6524,00AK,small_airport,Lowell Field,450.0,NA,US,US-AK,Anchor Point,NaN,Anc
3,6525,00AL,small_airport,Epps Airpark,820.0,NA,US,US-AL,Harvest,NaN,Har
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NA,US,US-AR,Newport,NaN,New


In [81]:
airports.drop(['abbrevation'], axis=1, inplace = True)

## JOIN

You need to provide which columns to join on (left_on and right_on), and join type: inner (default), left (corresponds to LEFT OUTER in SQL), right (RIGHT OUTER), or outer (FULL OUTER).

|                                                                               SQL                                                                              |                                                                                    Pandas                                                                                    |
|:--------------------------------------------------------------------------------------------------------------------------------------------------------------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| select airport_ident, type, description, frequency_mhz from airport_freq join airports on airport_freq.airport_ref = airports.id where airports.ident = 'KLAX' | pd.merge(left_table, right_table, left_on='airport_ref', right_on='id', suffixes =['-left','-right'] ,how='inner')|

In [82]:
pd.merge(airport_freq, airports[airports.ident == 'KLAX'][['id']],
         left_on='airport_ref', right_on='id', how='inner')[
    ['airport_ident', 'type', 'description', 'frequency_mhz']].head()

,airport_ident,type,description,frequency_mhz
0,KLAX,APP,SOCAL APP,36.07
1,KLAX,APP,SOCAL APP,124.30
2,KLAX,ATIS,ATIS,133.80
3,KLAX,CLD,CLNC DEL,121.40
4,KLAX,DEP,SOCAL DEP,124.30


## Join with condition

Another common type of join is based on condition rather than equal. For exampple, "date" > to_date('2018-10-01') and "date" <= to_date('2018-10-07')

2 solutions:
1. join without this condition, and filter on the merged data
2. populate all the values from 2018-10-01 to 2018-10-07, loop through and concat the results 

## UNION ALL and UNION
Use pd.concat() to UNION ALL two dataframes  
To deduplicate things (equivalent of UNION), you’d also have to add **.drop_duplicates()**.  
axis : {0/’index’, 1/’columns’}, default 0

|                     SQL     |               Pandas    |
|:---------------------------:|:-----------------------:|
| select name, municipality from airports where ident = 'KLAX' union all select name, municipality from airports where ident = 'KLGB' | pd.concat([airports[airports.ident == 'KLAX'][['name', 'municipality']], airports[airports.ident == 'KLGB'][['name', 'municipality']]]) |

In [83]:
pd.concat([airports[airports.ident == 'KLAX'][['name', 'municipality']], 
           airports[airports.ident == 'KLGB'][['name', 'municipality']]]).drop_duplicates()

,name,municipality
27701,Los Angeles International Airport,Los Angeles
27726,Long Beach /Daugherty Field/ Airport,Long Beach


## Immutability

I need to mention one important thing — immutability. By default, most operators applied to a Pandas dataframe return a new object. Some operators accept a parameter inplace=True, so you can work with the original dataframe instead. For example, here is how you would reset an index in-place:

In [84]:
airports.reset_index(drop=True, inplace=True)

However, the **.loc** operator in the **UPDATE** example above simply locates indices of records to updates, and the values are changed in-place. Also, if you updated all values in a column:

# WINDOW FUNCTIONS

Window function contains 4 parts:
1. Function: will cover most common functions below:
    1. Value: first value, last value, lead, lag...
    2. Calculation: rolling sum, cumsum...
    3. Ranking: row_number(), percentile()... 
2. group by: .groupby()
3. Order by: optional, specify an order for calculation. by timestampe, value etc...
4. Frame: optional, range of this calculation. After defining the order, specify window starts at the certain rows of the partition and ends at certain rows. 

For example, 
  for each city(**group by**), the moving average(**function**) of daily water usage for previous 7 days(**frame**) sorted by date(**order**)  





## ROW_NUMBER 

We want to calcute some rank partitioned by group and sort by a value.  

**.assign()** to create new columns to a DataFrame, returning a new object (a copy) with the new columns added to the original ones.   
**.rank(method, ascending)** to calculate the rank. There are different method in case of a tie:
- average: average rank of group
- min: lowest rank in group
- max: highest rank in group
- first: ranks assigned in order they appear in the array 
**.query('condition')** to filter out the rows doesn't qualify 



|                     SQL     |               Pandas    |
|:---------------------------:|:-----------------------:|
| ROW_NUMBER() OVER(PARTITION BY continent ORDER BY elevation_ft DESC) AS rn ... where rn<3 | airports.assign(rn = airports.groupby(['continent'])['elevation_ft'].rank(method='first', ascending=False)).query('rn <3') |

In [86]:
airports.assign(rn = airports.groupby(['continent'])['elevation_ft']
                .rank(method='first', ascending=False)).query('rn <3').sort_values('continent')

,id,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,home_link,rn
21320,30075,FXLT,small_airport,Letseng Airport,10400.0,AF,LS,LS-J,Letseng,NaN,1.0
26598,42694,KE-0051,small_airport,Rutundu Airport,10200.0,AF,KE,KE-400,Rutundu,NaN,2.0
18366,324046,EGAT,small_airport,Sky-Blu Field Station,4740.0,AN,AQ,AQ-U-A,NaN,https://www.bas.ac.uk/polar-operations/sites-a...,2.0
36810,5054,NZSP,medium_airport,South Pole Station Airport,9300.0,AN,AQ,AQ-U-A,NaN,NaN,1.0
23936,35129,IN-0001,heliport,Siachen Glacier AFS Airport,22000.0,AS,IN,IN-JK,NaN,NaN,1.0
23938,42716,IN-0003,small_airport,Daulat Beg Oldi Advanced Landing Ground,16200.0,AS,IN,IN-JK,NaN,NaN,2.0
14477,324871,CH-0006,heliport,Helipad Station Corvatsch,10837.0,EU,CH,CH-GR,Silvaplana,NaN,2.0
21043,43610,FR-0268,small_airport,Dome De La Lauze Airport,11647.0,EU,FR,FR-V,La Grave,NaN,1.0
14064,16652,CD21,heliport,Badger Mountain Heliport,11294.0,NA,US,US-CO,Tarryall,NaN,2.0
14080,16668,CD37,heliport,Berthoud Pass Heliport,12442.0,NA,US,US-CO,Empire,NaN,1.0


## LEAD, LAG

In [17]:
# sort the data by desired order
airports.sort_values(['continent','elevation_ft'], ascending = [False,False], inplace=True)

airports["elevation_ft_lag"] = airports.groupby(['continent'])["elevation_ft"].shift(1)
airports.head(4)

,id,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,home_link,elevation_ft_lag
45425,6211,SPGB,small_airport,Galilea Airport,14965.0,SA,PE,PE-AMA,NaN,NaN,NaN
45505,39647,SPRG,small_airport,San Regis Airport,14809.0,SA,PE,PE-ICA,Chincha,NaN,14965.0
44565,39473,SLUY,small_airport,Uyuni Airport,14422.0,SA,BO,BO-P,Quijarro,NaN,14809.0
45478,32362,SPNT,small_airport,Intuto Airport,14360.0,SA,PE,PE-LOR,Intuto,NaN,14422.0


In [18]:
airports["elevation_ft_lead"] = airports.groupby(['continent'])["elevation_ft"].shift(-1)
airports.head(4)

,id,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,home_link,elevation_ft_lag,elevation_ft_lead
45425,6211,SPGB,small_airport,Galilea Airport,14965.0,SA,PE,PE-AMA,NaN,NaN,NaN,14809.0
45505,39647,SPRG,small_airport,San Regis Airport,14809.0,SA,PE,PE-ICA,Chincha,NaN,14965.0,14422.0
44565,39473,SLUY,small_airport,Uyuni Airport,14422.0,SA,BO,BO-P,Quijarro,NaN,14809.0,14360.0
45478,32362,SPNT,small_airport,Intuto Airport,14360.0,SA,PE,PE-LOR,Intuto,NaN,14422.0,13720.0


## CUMSUM and Rolling Sum


In [ ]:
df["running_total_trips"] = df.trips.cumsum()

In [19]:
airports.continent.unique()

array(['SA', 'OC', 'NA', 'EU', 'AS', 'AN', 'AF'], dtype=object)